In [1]:
from dotenv import load_dotenv
import os
import base64

In [2]:
load_dotenv()

True

In [3]:
LANGFUSE_PUBLIC_KEY= os.getenv('LANGFUSE_PUBLIC_KEY')
LANGFUSE_SECRET_KEY= os.getenv('LANGFUSE_SECRET_KEY')
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:3000/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# your Hugging Face token
os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

In [4]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

/Users/niteshkumarsharma/Desktop/Folder/AI/agent-orchestra/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    DuckDuckGoSearchTool,
    VisitWebpageTool,
    HfApiModel,
)
 
model = HfApiModel(
    model_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
)
 
search_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="This is an agent that can do web search.",
)
 
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[search_agent],
)
manager_agent.run(
    "How to mango shake?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How to make simple pasta recipe?                                                                                │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1-Distill-Qwen-32B ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Simple Spaghetti with Tomato and Garlic Sauce Recipe                                                           
                                                                                                                   
  # Ingredients                                                                                                    
  pasta = "400g spaghetti"                                                                                         
  water = "4 quarts"                                                                                               
  salt = "1 teaspoon"                                                                                              
  olive_oil = "1 tablespoon"                                                                                       
  garlic = "1 clove, minced"                                                                                       
  tomatoes = "1 small can chopped tomatoes"                                                                        
  sauce_salt = "1/2 teaspoon"                                                                                      
  basil = "fresh, chopped (optional)"                                                                              
  black_pepper = "to taste"                                                                                        
  parmesan = " grated (optional)"                                                                                  
                                                                                                                   
  # Instructions                                                                                                   
  print("Step 1: Boil the Water\n")                                                                                
  print(f"Bring {water} to a boil in a large pot. Add {salt} and {pasta}. Stir occasionally.\n")                   
                                                                                                                   
  print("Step 2: Cook the Pasta\n")                                                                                
  print("Cook for 8-10 minutes until al dente.\n")                                                                 
                                                                                                                   
  print("Step 3: Prepare the Sauce\n")                                                                             
  print(f"Heat {olive_oil} in a skillet over medium heat. Add {garlic} and sauté until fragrant.\n")               
  print(f"Add {tomatoes} and {sauce_salt}. Stir in {basil} or oregano if using. Cook for 10 minutes until          
  thickened.\n")                                                                                                   
                                                                                                                   
  print("Step 4: Combine Pasta and Sauce\n")                                                                       
  print("Drain pasta, add to sauce. Toss to coat. Add {black_pepper} to taste.\n")                                 
                                                                                                                   
  print("Step 5: Serve\n")                                                                                         
  print(f"Serve warm. Sprinkle with {parmesan} if desired.\n")                                                     
                                                                                                                   
  print("Enjoy your simple spaghetti with tomato and garlic sauce!\n")                                             
 ────────────────────────────────────────────────────────

Execution logs:
Step 1: Boil the Water

Bring 4 quarts to a boil in a large pot. Add 1 teaspoon and 400g spaghetti. Stir occasionally.

Step 2: Cook the Pasta

Cook for 8-10 minutes until al dente.

Step 3: Prepare the Sauce

Heat 1 tablespoon in a skillet over medium heat. Add 1 clove, minced and sauté until fragrant.

Add 1 small can chopped tomatoes and 1/2 teaspoon. Stir in fresh, chopped (optional) or oregano if using. Cook for 
10 minutes until thickened.

Step 4: Combine Pasta and Sauce

Drain pasta, add to sauce. Toss to coat. Add {black_pepper} to taste.

Step 5: Serve

Serve warm. Sprinkle with  grated (optional) if desired.

Enjoy your simple spaghetti with tomato and garlic sauce!


Out: None

[Step 0: Duration 54.07 seconds| Input tokens: 2,098 | Output tokens: 967]

...

Error in generating model output:
500 Server Error: Internal Server Error for url: 
https://router.huggingface.co/hf-inference/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B/v1/chat/completions 
(Request ID: Root=1-67c5be55-74ea131d7e10bcf6099c31de;ea086188-dd49-43c8-b4a4-121ebcb001e6)

Model too busy, unable to get response in less than 60 second(s)

[Step 5: Duration 60.40 seconds| Input tokens: 30,526 | Output tokens: 5,131]

Reached max steps.

[Step 6: Duration 60.40 seconds| Input tokens: 37,347 | Output tokens: 6,238]

"Okay, so I need to figure out how to make a simple pasta recipe. Let's start by thinking about what pasta is and how it's typically prepared. Pasta is a versatile dish that can be made in many ways, but since the user asked for something simple, I should keep it straightforward.\n\nFirst, I'll think about the basic components of pasta. There's the pasta itself, like spaghetti, penne, or fettuccine, and then the sauce. For simplicity, maybe a tomato-based sauce would be best because it's classic and doesn't require too many ingredients.\n\nNow, considering the ingredients, I'll need pasta, water, salt for cooking the pasta, and then olive oil, garlic, and canned tomatoes for the sauce. Optional additions could be herbs like basil or oregano, salt and pepper for seasoning, and maybe some grated cheese like Parmesan for serving.\n\nNext, I'll break down the steps:\n\n1. **Boiling the Water:** This is the first step in cooking pasta. It's important to use enough water so the pasta doesn't